In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
movies=pd.read_csv("Desktop//tmdb_5000_movies.csv")
credits=pd.read_csv("Desktop//tmdb_5000_credits.csv")

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head()

In [ ]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.head()
movies.isnull().sum()
movies.dropna(inplace=True)

In [ ]:
import ast
def convert_list(text):
    L=[]
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L    

In [ ]:
movies['genres']=movies['genres'].apply(convert_list)
movies.head()

In [ ]:
movies['keywords'][0]

In [ ]:
movies['keywords']=movies['keywords'].apply(convert_list)
movies.head()

In [ ]:
movies['keywords'][0]

In [ ]:
def cast(text):
    L=[]
    c=0
    for i in ast.literal_eval(text):
        if c<=3:
            L.append(i['name'])
            c=c+1
        
    return L  

In [ ]:
movies['cast']=movies['cast'].apply(cast)


In [ ]:
movies['cast']

In [ ]:
def crew(text):
    L=[]
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            L.append(i['name'])
            
        
    return L

In [ ]:
movies['crew']=movies['crew'].apply(crew)

In [ ]:
movies['crew']

In [ ]:
movies.head()

In [ ]:
def remove_space(text):
    l=[]
    for i in text:
        l.append(i.replace(" ",""))
    return l    

In [ ]:
movies['genres']=movies['genres'].apply(remove_space)
movies['keywords']=movies['keywords'].apply(remove_space)
movies['cast']=movies['cast'].apply(remove_space)
movies['crew']=movies['crew'].apply(remove_space)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['overview'][0]

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [ ]:
movies['tags']=movies['tags'].apply(lambda x: " ".join(x))
movies['tags'][0]

In [ ]:
import nltk


In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
movies['tags']=movies['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
data=cv.fit_transform(movies['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similar_movies=cosine_similarity(data)

In [ ]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similar_movies[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(movies.iloc[i[0]].title)


In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(movies.to_dict(),open("movies_list2.pkl",'wb'))


In [ ]:
pickle.dump(similar_movies,open('similar_movies_list.pkl','wb'))